# Graph Traversals

This notebook assumes you have `simple_example.grg`, which can be constructed by running the `01.Intro.ipynb` notebook.

The goal is to demonstrate the different between the different graph traversals.

In [ ]:
from pygrgl import GRG, load_immutable_grg, get_dfs_order, get_bfs_order, get_topo_order, TraversalDirection
from pygrgl.display import grg_to_cyto
from IPython.display import display, SVG

GRG_FILENAME = "simple_example.grg"

Our graph is shown below. Lets consider two nodes in particular to demonstrate the traversal APIs: `nodeId=10` and `nodeId=3`.

In [ ]:
g = load_immutable_grg(GRG_FILENAME)
display(grg_to_cyto(g, show_mutations=False))

# Difference between DFS and Topological

First the case where the depth-first search (DFS) and topological order are the same. Let's do a traversal that starts at nodes `0, 3` and traverses upward. We expect the following nodes to be encountered: `0, 3, 8, 9, 10`. The topological order ensures that we _must_ encountered node `9` before we encounter node `10` -- even though if we started from node `0` and naively searched upward we might reach node `10` before `9`.

First we do the topological order, which is pretty intuitive:

In [ ]:
list(get_topo_order(g, TraversalDirection.UP, [0, 3]))

Next, in order to get the "equivalent" DFS order, we have to start from the _roots_ and search down. This is because the DFS imposes a similar order via a LIFO (stack). Another way to put this: if you do a DFS over the whole graph, and save the order that nodes are visited in for the second visit, this is the same order that the topological order provides. In fact this is how `get_topo_order` is implemented, it numbers the nodes via DFS and then uses a heap to allow traversing a subset of those nodes.

In [ ]:
list(get_dfs_order(g, TraversalDirection.DOWN, [10, 8]))

Notice how we visited `[0, 3, 8, 9, 10]` in the same order for DFS and topological, but that the DFS visited more nodes. This is because we had to provide the "parent" seeds and visit everything beneath them, whereas with the topological order we provided the "child" seeds and visited only nodes reachable upwards.

### Equivalent DFS and Topological

Now we demonstrate the topological property is maintained for both.

In [ ]:
visited = [False] * g.num_nodes
topo_nodes = list(get_topo_order(g, TraversalDirection.UP, g.get_sample_nodes()))
for node_id in topo_nodes:
    for child_id in g.get_down_edges(node_id):
        assert visited[child_id]
    visited[node_id] = True
print(f"Topoological property holds for {topo_nodes}")

In [ ]:
visited = [False] * g.num_nodes
dfs_nodes = list(get_dfs_order(g, TraversalDirection.DOWN, g.get_root_nodes()))
for node_id in dfs_nodes:
    for child_id in g.get_down_edges(node_id):
        assert visited[child_id]
    visited[node_id] = True
print(f"Topoological property holds for {dfs_nodes}")

You'll notice that the topological ordering of all the nodes is exactly the same and `range(0, g.num_nodes)`. This is by design, as the GRG is renumbered (when written to disk) specifically to maintain this property.

## Breadth-first

The breadth-first search (BFS) enumerates the nodes such that siblings are iterated prior to descendants. Below we compare a `forward_only` DFS with a BFS, both starting at node `10` and traversing downwards. A "regular" DFS visits nodes in a post-order fashion: a node is only visited after all of its children have (recursively) been visited. A `forward_only` DFS visits nodes in a pre-order fashion: a node is visited prior to any of its children. This is a better comparison against a BFS, since they are both in a sense "pre-order".

### 

In [ ]:
list(get_dfs_order(g, TraversalDirection.DOWN, [10], forward_only=True))

In [ ]:
list(get_bfs_order(g, TraversalDirection.DOWN, [10]))

Notice that the BFS order visits all of `10`s children prior to visited its grand-children. The DFS on the other hand does not have this property.